In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-google-genai python-dotenv pyvis

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata


os.environ['GOOGLE_API_KEY']=userdata.get("GOOGLE_API_KEY")
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

In [13]:
llm.invoke("what is knowlede graph")

AIMessage(content='A **Knowledge Graph** is a structured way to represent information about real-world entities and their relationships, in a format that is easily understandable by both humans and machines. Think of it as a highly interconnected, semantic network of facts.\n\nIt goes beyond traditional databases by not just storing data, but also capturing the *meaning* and *relationships* between different pieces of information.\n\n### Core Components of a Knowledge Graph:\n\n1.  **Entities (Nodes):** These are the "things" or "concepts" in the real world.\n    *   **Examples:** People (Albert Einstein), Places (Berlin), Organizations (NASA), Events (World War II), Concepts (Relativity Theory), Products (iPhone).\n\n2.  **Relationships (Edges/Predicates):** These describe how entities are connected to each other. Relationships are directional.\n    *   **Examples:** "was born in," "works for," "is a type of," "wrote," "discovered."\n\n3.  **Attributes/Properties:** These are characte

In [14]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document

graph_transformer=LLMGraphTransformer(llm=llm)

#extract graph from text

In [15]:
data="""Iron Man is a superhero appearing in American comic books published by Marvel Comics.
Co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby,
the character first appeared in Tales of Suspense #39 in 1962 (cover dated March 1963) and received his own title with Iron Man #1 in 1968.
 Shortly after his creation, Iron Man became a founding member of the superhero team, the Avengers, alongside Thor, Ant-Man, the Wasp, and the Hulk.
Iron Man stories, individually and with the Avengers, have been published consistently since the character's creation.
"""

In [16]:
documents = [Document(page_content=data)]
graph=await  graph_transformer.aconvert_to_graph_documents(documents)

In [17]:
graph

[GraphDocument(nodes=[Node(id='Iron Man', type='Character', properties={}), Node(id='Marvel Comics', type='Organization', properties={}), Node(id='Stan Lee', type='Person', properties={}), Node(id='Larry Lieber', type='Person', properties={}), Node(id='Don Heck', type='Person', properties={}), Node(id='Jack Kirby', type='Person', properties={}), Node(id='Tales Of Suspense #39', type='Comic book', properties={}), Node(id='1962', type='Year', properties={}), Node(id='March 1963', type='Date', properties={}), Node(id='Iron Man #1', type='Comic book', properties={}), Node(id='1968', type='Year', properties={}), Node(id='The Avengers', type='Team', properties={}), Node(id='Thor', type='Character', properties={}), Node(id='Ant-Man', type='Character', properties={}), Node(id='The Wasp', type='Character', properties={}), Node(id='The Hulk', type='Character', properties={})], relationships=[Relationship(source=Node(id='Iron Man', type='Character', properties={}), target=Node(id='Marvel Comics',

In [18]:
print(f"Nodes:{graph[0].nodes}")
print(f"Relationships:{graph[0].relationships}")

Nodes:[Node(id='Iron Man', type='Character', properties={}), Node(id='Marvel Comics', type='Organization', properties={}), Node(id='Stan Lee', type='Person', properties={}), Node(id='Larry Lieber', type='Person', properties={}), Node(id='Don Heck', type='Person', properties={}), Node(id='Jack Kirby', type='Person', properties={}), Node(id='Tales Of Suspense #39', type='Comic book', properties={}), Node(id='1962', type='Year', properties={}), Node(id='March 1963', type='Date', properties={}), Node(id='Iron Man #1', type='Comic book', properties={}), Node(id='1968', type='Year', properties={}), Node(id='The Avengers', type='Team', properties={}), Node(id='Thor', type='Character', properties={}), Node(id='Ant-Man', type='Character', properties={}), Node(id='The Wasp', type='Character', properties={}), Node(id='The Hulk', type='Character', properties={})]
Relationships:[Relationship(source=Node(id='Iron Man', type='Character', properties={}), target=Node(id='Marvel Comics', type='Organizat

In [31]:
from pyvis.network import Network

def visualize_graph(graph_documents,output_file="knowledge_graph.html"):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")

    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}

    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error
    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)


    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")


In [21]:
visualize_graph(graph)

Graph saved to /content/knowledge_graph.html


#from .txt file to kg

In [25]:
file="/content/dummy_graph_data.txt"


with open(file,"r") as f:
  data=f.read()


In [27]:
print(data)

Document1 -> TopicA
Document1 -> TopicB
Document2 -> TopicB
Document2 -> TopicC
Document3 -> TopicA
Document3 -> TopicC
TopicA -> TopicB
TopicB -> TopicC
TopicC -> TopicD



In [29]:
document=[Document(page_content=data)]
graph_doc=await  graph_transformer.aconvert_to_graph_documents(document)

In [30]:
graph_doc

[GraphDocument(nodes=[Node(id='Document1', type='Document', properties={}), Node(id='Topica', type='Topic', properties={}), Node(id='Topicb', type='Topic', properties={}), Node(id='Document2', type='Document', properties={}), Node(id='Topicc', type='Topic', properties={}), Node(id='Document3', type='Document', properties={}), Node(id='Topicd', type='Topic', properties={})], relationships=[Relationship(source=Node(id='Document1', type='Document', properties={}), target=Node(id='Topica', type='Topic', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='Document1', type='Document', properties={}), target=Node(id='Topicb', type='Topic', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='Document2', type='Document', properties={}), target=Node(id='Topicb', type='Topic', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='Document2', type='Document', properties={}), target=Node(id='Topicc', type='Topic', pro

In [32]:
visualize_graph(graph_doc,"output.html")

Graph saved to /content/output.html
